In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'

# Debt Market Model

The purpose of this notebook is to configure and simulate the full CDP and APT system model, using the historical Ethereum price as a driver, under different PI controller settings - enabled, disabled, `kp` and `ki`.

# Setup and Dependencies

In [2]:
# Set project root folder, to enable importing project files from subdirectories
from pathlib import Path
import os

path = Path().resolve()
root_path = str(path).split('notebooks')[0]
os.chdir(root_path)

# Force reload of project modules, sometimes necessary for Jupyter kernel
%load_ext autoreload
%autoreload 2

# Display cadCAD version for easy debugging
%pip show cadCAD

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /Users/bscholtz/workspace/reflexer/venv/lib/python3.8/site-packages
Requires: pathos, funcy, pandas, fn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import all shared dependencies and setup
from shared import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import plotly.io as pio
# pio.renderers.default = "png"

# Historical MakerDAO Dai debt market activity

In [4]:
# Import the historical MakerDAO market data CSV file
from models.system_model_v3.model.params.init import debt_market_df
debt_market_df

,seconds_passed,rho_star,beta,p,Q,v_1,v_2 + v_3,u_1,u_2 + u_3,u_2,...,u_3,w_3,w_2 + w_3,D_1,D_2,w_1,D,C_star,p_star,p_ema_10
date,,,,,,,,,,,,,,,,,,,,,
2018-04-01,86400,383.822378,0.005,0.996472,1.304176e+05,3575.828421,75.286089,397538.457500,3.369517e+04,3.346177e+04,...,2.334048e+02,4.713235e-02,6.804199,1.937743e+07,9.035786e+03,264.904620,1.938647e+07,5.005718e+07,1,1.001429
2018-04-02,86400,386.787115,0.005,0.999184,1.344705e+05,4158.239991,105.338665,206636.000000,1.195705e+04,1.195705e+04,...,2.915840e-09,4.652248e-13,1.907757,1.957211e+07,9.301446e+03,267.568355,1.958141e+07,5.201145e+07,1,1.001174
2018-04-03,86400,402.818424,0.005,0.993012,1.347840e+05,976.262655,662.782265,321120.330000,2.221112e+05,2.221112e+05,...,0.000000e+00,0.000000e+00,128.782844,1.967112e+07,9.441589e+03,268.924904,1.968056e+07,5.429346e+07,1,1.001308
2018-04-04,86400,392.545993,0.005,0.989616,1.364970e+05,1781.667763,68.592053,217219.126161,2.063823e+05,2.063823e+05,...,3.492460e-10,2.725929e-13,161.085196,1.968196e+07,9.549578e+03,269.074899,1.969151e+07,5.358136e+07,1,1.001366
2018-04-05,86400,379.389608,0.005,1.002888,1.378249e+05,1449.263202,121.398587,411720.500000,4.433695e+04,4.433695e+04,...,1.862645e-09,2.339915e-13,5.569751,2.004934e+07,9.818105e+03,274.096521,2.005916e+07,5.228933e+07,1,1.002515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-26,86400,167.637314,0.125,1.021405,1.525944e+06,56696.770732,29566.578527,807916.984507,1.585480e+06,1.515115e+06,...,7.036438e+04,2.984212e+03,67241.524162,7.957442e+07,4.562428e+06,27167.708945,8.413685e+07,2.558052e+08,1,1.008076
2019-09-27,86400,167.347085,0.125,1.007346,1.529959e+06,7598.158072,3582.812664,608760.677317,6.367461e+05,6.366783e+05,...,6.779026e+01,7.681557e-01,7215.198255,7.954644e+07,4.582358e+06,27145.743151,8.412880e+07,2.560342e+08,1,1.009903
2019-09-28,86400,174.289775,0.125,1.011006,1.528450e+06,876.961579,2386.452089,225195.561593,1.452312e+05,1.452062e+05,...,2.500000e+01,8.480860e-02,492.674170,7.962640e+07,4.609044e+06,27177.983765,8.423545e+07,2.663932e+08,1,1.011184


In [5]:
# Plot the full set of historical data over time
debt_market_df.plot()

# Model Configuration

In [6]:
from models.system_model_v3.model.state_variables.init import state_variables

state_variables.update({})


130417.5685990905
189872.3149593018
29727.37318010564
29727.37318010564


19377432.0069828
42320205.44783997
16068921.349645888
6873852.091211276

130417.5685990905 33371454.249511395


In [7]:
from models.system_model_v3.model.params.init import params

# Update the default parameter values
params_update = {
    'controller_enabled': [True],
    'kp': [1e-7], # 5e-7 # proportional term for the stability controller: units 1/USD
    'ki': [lambda control_period=3600: -1e-7 / control_period], # integral term for the stability controller: units 1/(USD*seconds)
}

params.update(params_update)

# Simulation Execution

In [30]:
# Set the number of simulation timesteps, with a maximum of `len(debt_market_df) - 1`
SIMULATION_TIMESTEPS = 100 # len(debt_market_df) - 1

In [31]:
# Create a wrapper for the model simulation, and update the existing parameters and initial state
system_simulation = ConfigWrapper(system_model_v3, T=range(SIMULATION_TIMESTEPS), M=params, initial_state=state_variables)

In [32]:
del configs[:] # Clear any prior configs
system_simulation.append() # Append the simulation config to the cadCAD `configs` list
(simulation_result, _tensor_field, _sessions) = run(drop_midsteps=False) # Run the simulation


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100, 33, 1, 46)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [5]
Execution Mode: single_threaded
Performing arb. UNI -> CDP
1.496320241602279
0
UniswapObservation(timestamp=86400.0, price_0_cumulative=336.82785612718857, price_1_cumulative=22162537.522375163)
ConverterFeedObservation(timestamp=86400.0, time_adjusted_price=33162253.49999998)
1.5078781780408017
0
UniswapObservation(timestamp=172800.0, price_0_cumulative=673.6557122543771, price_1_cumulative=44325075.044750325)
ConverterFeedObservation(timestamp=172800.0, time_adjusted_price=33418406.69999997)
Over collateralized?
Performing arb. CDP -> UNI
1.501190686662542

# Simulation Analysis

In [33]:
# Add new columns to dataframe
simulation_result = simulation_result.assign(eth_collateral_value = simulation_result.eth_collateral * simulation_result.eth_price)
simulation_result['collateralization_ratio'] = (simulation_result.eth_collateral * simulation_result.eth_price) / (simulation_result.principal_debt * simulation_result.target_price)

# Update dataframe display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

simulation_result

,index,events,cdp_metrics,optimal_values,timedelta,cumulative_time,timestamp,blockheight,eth_price,cdps,eth_collateral,eth_locked,eth_freed,eth_bitten,v_1,v_2,v_3,principal_debt,rai_drawn,rai_wiped,rai_bitten,u_1,u_2,u_3,accrued_interest,interest_dripped,interest_wiped,interest_bitten,w_1,w_2,w_3,system_revenue,stability_fee,market_price,market_price_twap,target_price,target_rate,eth_return,eth_gross_return,expected_market_price,expected_debt_price,error_star,error_star_integral,uniswap_oracle,RAI_balance,ETH_balance,UNI_supply,simulation,subset,run,substep,timestep,eth_collateral_value,collateralization_ratio
0,0,None,{},{},0.0,0.0,2018-04-01,0,383.822378,None,130417.568599,130417.568599,0.000000,0.0,0.0,0.0,0.0,3.337145e+07,3.337145e+07,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0.000000,0.0,0.0,0.0,1.585490e-10,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.0,1.000000,1.0,0.000000,0.000000,<models.system_model_v3.model.parts.uniswap_or...,5.000000e+06,13026.858986,5000000.0,0,0,1,0,0,5.005718e+07,1.500000
1,1,None,{},{},0.0,0.0,2018-04-01,0,383.822378,None,130417.568599,130417.568599,0.000000,0.0,0.0,0.0,0.0,3.337145e+07,3.337145e+07,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0.000000,0.0,0.0,0.0,1.585490e-10,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.0,1.000000,1.0,0.000000,0.000000,<models.system_model_v3.model.parts.uniswap_or...,5.000000e+06,13026.858986,5000000.0,0,0,1,1,1,5.005718e+07,1.500000
2,2,None,{},{},86400.0,86400.0,2018-04-02,0,383.822378,None,130417.568599,130417.568599,0.000000,0.0,0.0,0.0,0.0,3.337145e+07,3.337145e+07,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0.000000,0.0,0.0,0.0,1.585490e-10,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.0,1.000000,1.0,0.000000,0.000000,<models.system_model_v3.model.parts.uniswap_or...,5.000000e+06,13026.858986,5000000.0,0,0,1,2,1,5.005718e+07,1.500000
3,3,None,{},{},86400.0,86400.0,2018-04-02,0,383.822378,None,130417.568599,130417.568599,0.000000,0.0,0.0,0.0,0.0,3.337145e+07,3.337145e+07,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0.000000,0.0,0.0,0.0,1.585490e-10,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.0,1.000209,1.0,0.000000,0.000000,<models.system_model_v3.model.parts.uniswap_or...,5.000000e+06,13026.858986,5000000.0,0,0,1,3,1,5.005718e+07,1.500000
4,4,None,{},{},86400.0,86400.0,2018-04-02,0,383.822378,None,130417.568599,130417.568599,0.000000,0.0,0.0,0.0,0.0,3.337145e+07,3.337145e+07,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0.000000,0.0,0.0,0.0,1.585490e-10,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.0,1.000209,1.0,0.000000,0.000000,<models.system_model_v3.model.parts.uniswap_or...,4.088620e+06,15939.366412,5000000.0,0,0,1,4,1,5.005718e+07,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1496,1496,[],"{'cdp_count': 1, 'open_cdp_count': 1, 'closed_...",{},86400.0,8640000.0,2018-07-10,0,484.287218,open time locked drawn ...,126855.844703,130417.568599,3561.723896,0.0,0.0,0.0,0.0,2.576241e+07,6.995095e+07,4.418854e+07,0.0,0.0,307239.654804,0.0,35549.027045,0,0,0,18.220445,0.0,0.0,0.0,1.585490e-10,1.686607,1.668855,1.608964,-8.634172e-08,-0.019269,1.0,0.883157,1.0,-0.098734,66068.591697,<models.system_model_v3.model.parts.uniswap_or...,4.326181e+06,15066.613089,5000000.0,0,0,1,11,100,6.143466e+07,1.482111
1497,1497,[],"{'cdp_count': 1, 'open_cdp_count': 1, 'closed_...",{},86400.0,8640000.0,2018-07-10,0,484.287218,open time locked drawn ...,126855.844703,130417.568599,3561.723896,0.0,0.0,0.0,0.0,2.576241e+07,6.995095e+07,4.449577e+07,0.0,0.0,307239.654804,0.0,35549.027045,0,0,0,18.220445,0.0,0.0,0.0,1.585490e-10,1.686607,1.668855,1.608964,-8.634172e-08,-0.019269,1.0,0.883157,1.0,-0.098734,66068.591697,<models.system_model_v3.model.parts.uniswap_or...,4.326181e+06,15066.613089,5000000.0,0,0,1,12,100,6.143466e+07,1.482111
1498,1498,

## Save simulation

In [34]:
# Save the simulation result to a pickle file, for backup - this will be overwritten on the next simulation
simulation_result.to_pickle(f'./exports/system_model_v3/results.pickle')

# Simulation Analysis

In [35]:
# Load the simulation result from a pickle file, specifying past results when necessary
simulation_result = pd.read_pickle(f'exports/system_model_v3/results.pickle')

# Drop the simulation midsteps - the substeps that aren't used for generating plots
df = drop_dataframe_midsteps(simulation_result)
df

,level_0,index,events,cdp_metrics,optimal_values,timedelta,cumulative_time,timestamp,blockheight,eth_price,cdps,eth_collateral,eth_locked,eth_freed,eth_bitten,v_1,v_2,v_3,principal_debt,rai_drawn,rai_wiped,rai_bitten,u_1,u_2,u_3,accrued_interest,interest_dripped,interest_wiped,interest_bitten,w_1,w_2,w_3,system_revenue,stability_fee,market_price,market_price_twap,target_price,target_rate,eth_return,eth_gross_return,expected_market_price,expected_debt_price,error_star,error_star_integral,uniswap_oracle,RAI_balance,ETH_balance,UNI_supply,simulation,subset,run,substep,timestep,eth_collateral_value,collateralization_ratio
0,0,0,None,{},{},0.0,0.0,2018-04-01,0,383.822378,None,130417.568599,130417.568599,0.000000,0.0,0.0,0.000000,0.0,3.337145e+07,3.337145e+07,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0,0,0,0.000000,0.0,0.0,0.0,1.585490e-10,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.0,1.000000,1.0,0.000000,0.000000,<models.system_model_v3.model.parts.uniswap_or...,5.000000e+06,13026.858986,5000000.0,0,0,1,0,0,5.005718e+07,1.500000
1,15,15,None,"{'cdp_count': 1, 'open_cdp_count': 1, 'closed_...",{},86400.0,86400.0,2018-04-02,0,386.787115,None,126855.844703,130417.568599,3561.723896,0.0,0.0,3561.723896,0.0,3.246007e+07,3.337145e+07,9.113796e+05,0.0,0.000000e+00,9.113796e+05,0.0,457.146316,0,0,0,457.146316,0.0,0.0,0.0,1.585490e-10,1.496320,0.000000,1.000000,0.000000e+00,0.007724,1.0,1.000209,1.0,0.000000,0.000000,<models.system_model_v3.model.parts.uniswap_or...,4.088620e+06,15939.366412,5000000.0,0,0,1,15,1,4.906621e+07,1.511586
2,30,30,None,"{'cdp_count': 1, 'open_cdp_count': 1, 'closed_...",{},86400.0,172800.0,2018-04-03,0,402.818424,None,126855.844703,130417.568599,3561.723896,0.0,0.0,0.000000,0.0,3.278101e+07,3.369239e+07,9.113796e+05,0.0,3.209400e+05,0.000000e+00,0.0,901.814159,0,0,0,0.006262,0.0,0.0,0.0,1.585490e-10,1.507878,0.000000,0.997858,-2.481601e-08,0.041447,1.0,0.994857,1.0,-0.496320,-21441.034437,<models.system_model_v3.model.parts.uniswap_or...,4.088620e+06,15939.366412,5000000.0,0,0,1,15,2,5.109987e+07,1.562171
3,45,45,None,"{'cdp_count': 1, 'open_cdp_count': 1, 'closed_...",{},86400.0,259200.0,2018-04-04,0,392.545993,None,126855.844703,130417.568599,3561.723896,0.0,0.0,0.000000,0.0,3.407547e+07,3.498685e+07,9.113796e+05,0.0,1.294455e+06,0.000000e+00,0.0,1350.884561,0,0,0,5.674483,0.0,0.0,0.0,1.585490e-10,1.501191,2.236902,0.999739,2.179649e-08,-0.025501,1.0,0.990673,1.0,-0.510020,-62897.897771,<models.system_model_v3.model.parts.uniswap_or...,4.181915e+06,15584.817693,5000000.0,0,0,1,15,3,4.979675e+07,1.461748
4,60,60,None,"{'cdp_count': 1, 'open_cdp_count': 1, 'closed_...",{},86400.0,345600.0,2018-04-05,0,379.389608,None,126855.844703,130417.568599,3561.723896,0.0,0.0,0.000000,0.0,3.301659e+07,3.498685e+07,1.970256e+06,0.0,0.000000e+00,1.058877e+06,0.0,1817.693494,0,0,0,16.454442,0.0,0.0,0.0,1.585490e-10,1.462908,1.975199,1.005489,6.638097e-08,-0.033516,1.0,0.993122,1.0,-0.501452,-100678.569819,<models.system_model_v3.model.parts.uniswap_or...,4.181915e+06,15584.817693,5000000.0,0,0,1,15,4,4.812779e+07,1.449727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1440,1440,None,"{'cdp_count': 1, 'open_cdp_count': 1, 'closed_...",{},86400.0,8294400.0,2018-07-06,0,471.660938,None,126855.844703,130417.568599,3561.723896,0.0,0.0,0.000000,0.0,2.410723e+07,6.829577e+07,4.418854e+07,0.0,5.317970e+05,0.000000e+00,0.0,34197.710792,0,0,0,0.108540,0.0,0.0,0.0,1.585490e-10,1.651398,1.622111,1.663461,-1.125727e-07,-0.005307,1.0,0.935570,1.0,0.048967,101493.498631,<models.system_model_v3.model.parts.uniswap_or...,4.326181e+06,15066.613089,5000000.0,0,0,1,15,96,5.983295e+07,1.492039
97,1455,1455,None,"{'cdp_count': 1, 'open_cdp_count': 1, 'closed_...",{},86400.0,8380800.0,2018-07-07,0,472.021399,None,126855.844703,130417.568599,3561.723896

## Select simulation

In [36]:
# Select the first simulation and subset, this is only relevant when running parameter sweeps or Monte Carlo Runs
# The following plots are configured for single simulation results
df = df.query('simulation == 0 and subset == 0')

In [37]:
df.plot(x='timestamp', y=['market_price', 'market_price_twap'])

In [15]:
df.plot(x='timestamp', y=['eth_price'], title='Historical ETH price')

In [16]:
df.plot(x='timestamp', y=['eth_return'], title='Historical ETH return')

In [17]:
df.plot(x='timestamp', y=['target_price', 'market_price'], title='Target Price vs. Market Price')

In [18]:
df.plot(x='timestamp', y=['market_price', 'expected_market_price'], title='Expected Market Price')

In [19]:
df.plot(x='timestamp', y=['target_rate'], title='Controller Target Rate')

In [20]:
df['locked - freed - bitten'] = df['eth_locked'] - df['eth_freed'] - df['eth_bitten']
df.plot(y=['eth_collateral', 'locked - freed - bitten'], title='Debt Market Locked ETH Collateral')

In [21]:
df.plot(x='timestamp', y=['eth_collateral_value'], title='Debt Market Locked ETH Collateral Value ($)')

In [22]:
df.plot(x='timestamp', y=['eth_locked', 'eth_freed', 'eth_bitten'], title='Debt Market ETH State')

In [23]:
df.plot(x='timestamp', y=['v_1', 'v_2', 'v_3'], title='Debt Market ETH Lock, Free, Bite Activity')

In [24]:
df['apt_v_1'] = df['optimal_values'].map(lambda v: v.get('v_1', 0))
df['apt_v_2'] = df['optimal_values'].map(lambda v: v.get('v_2 + v_3', 0))

df.plot(x='timestamp', y=['apt_v_1', 'apt_v_2'], title='Debt Market ETH APT Lock, Free Activity')

In [25]:
df['drawn - wiped - bitten'] = df['rai_drawn'] - df['rai_wiped'] - df['rai_bitten']
df.plot(x='timestamp', y=['principal_debt', 'drawn - wiped - bitten'], title='Debt Market RAI State')

In [26]:
df.plot(x='timestamp', y=['rai_drawn', 'rai_wiped', 'rai_bitten'], title='Debt Market RAI State')

In [27]:
df.plot(x='timestamp', y=['u_1', 'u_2', 'u_3'], title='Debt Market RAI Draw, Wipe, Bite Activity')

In [28]:
df['sum_apt_u_1'] = df['optimal_values'].map(lambda v: v.get('u_1', 0))
df['sum_apt_u_2'] = df['optimal_values'].map(lambda v: v.get('u_2', 0))

df.plot(x='timestamp', y=['sum_apt_u_1', 'sum_apt_u_2'], title='Debt Market RAI APT Lock, Free Activity')

In [29]:
df['diff_u_1_u_2'] = df['sum_apt_u_1'] - df['sum_apt_u_2']
df['diff_u_1_u_2'] = df['diff_u_1_u_2'].cumsum()

df.plot(x='timestamp', y=['diff_u_1_u_2'])

## Accrued interest and system revenue

In [30]:
df.plot(x='timestamp', y=['w_1', 'w_2', 'w_3'], title='Accrued Interest Activity')

In [31]:
df.plot(x='timestamp', y=['accrued_interest'], title='Accrued Interest')

In [32]:
df.plot(x='timestamp', y=['system_revenue'], title='System Revenue')

In [33]:
df.plot(x='timestamp', y=['collateralization_ratio'], title='Collateralization Ratio')

In [34]:
df.plot(x='timestamp', y='RAI_balance', title='Uniswap RAI balance')

In [35]:
df.plot(x='timestamp', y='ETH_balance', title='Uniswap ETH balance')

In [36]:
df.plot(x='timestamp', y='UNI_supply', title='Uniswap UNI balance')

In [37]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['target_price'], name="Target price"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['market_price'], name="Market price"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['eth_price'], name="ETH price"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Market and Target Price vs. ETH Price"
)

# Set x-axis title
fig.update_xaxes(title_text="Timestamp")

# Set y-axes titles
fig.update_yaxes(title_text="Market and target price ($)", secondary_y=False)
fig.update_yaxes(title_text="ETH price ($)", secondary_y=True)

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig.show()

In [38]:

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['target_price'], name="Redemption Price"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['market_price'], name="Market Price"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['target_rate'], name="Redemption Rate"),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text="Market Price, Redemption Price and Redemption Rate"
)
# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Price (USD)", secondary_y=False)
fig.update_yaxes(title_text="Redemption Rate (1n = 1e-9)", secondary_y=True)

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig.show()

## Simulation statistics

In [39]:
std_mkt = df['market_price'].rolling(7).std()
std_mkt.plot()

In [40]:
np.std(df['market_price'])

0.9181954040087899

In [41]:
err_m_t = df['market_price'] - df['target_price']
err_m_t.plot()

In [42]:
np.sqrt(abs(df['market_price'] - df['target_price']).mean())

0.6374840177966536

In [43]:
np.corrcoef(df['market_price'],df['eth_price'])

array([[ 1.        , -0.05165519],
       [-0.05165519,  1.        ]])

In [44]:
np.corrcoef(df['market_price'],df['target_price'])

array([[1.        , 0.85807905],
       [0.85807905, 1.        ]])

In [45]:
np.corrcoef(df['market_price'],df['target_rate'])

array([[1.        , 0.15259715],
       [0.15259715, 1.        ]])

In [46]:

df['market_price_rolling'] = df['market_price'].rolling(7).std()
fig = px.histogram(df, x="market_price_rolling", nbins=25)

fig.update_layout(
    title="7-Day Rolling Standard Deviation Histogram, Market Price (Controller On)",
    xaxis_title="Standard Deviation",
    yaxis_title="Frequency",
)

fig.show()

In [47]:

# std_mkt_without = res_without_controller[‘market_price’].rolling(7).std()
# std_mkt_with = res_with_controller[‘market_price’].rolling(7).std()
# df =pd.DataFrame(dict(
#     series=np.concatenate(([“With Controller”]*len(std_mkt_with), [“Without Controller”]*len(std_mkt_without))),
#     data  =np.concatenate((std_mkt_with,std_mkt_without))
# ))

# fig = df.hist(x=“data”, color=“series”, nbins=25, barmode=“overlay”,
#         labels={
#             ‘count’ : “Count”,
#             ‘data’ : “Std Dev”,
#             ‘series’ : “Simulation”
#         },
#         title=“Histogram, Standard Deviations of Market Price”)
# fig.show()